# Download do market price do Bitcoin

### Data: 10/04/2018

## Metodologia

1. Baixar dados de 4 exchanges do *Quandl*
2. Unir dados em um data frame
    1. Tratar dados
    2. Salvar/atualizar
3. Calcular a media global entre eles

### Fontes:
- https://blog.patricktriest.com/analyzing-cryptocurrencies-python/
- https://plot.ly/python/candlestick-charts/

In [1]:
import os
import numpy as np
import pandas as pd
import pickle # for serializing and de-serializing a Python object structure.
from datetime import datetime

import quandl

import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

## Funcao auxiliar para fazer download dos dados

In [2]:
def get_quandl_data(quandl_id):
    '''Download and save Quandl dataseries'''
    save_path = '{}.pkl'.format(quandl_id).replace('/','-')
    try:
        f = open(save_path, 'rb')
        df = pickle.load(f)   
        print('Loaded {} from cache'.format(quandl_id))
    except (OSError, IOError) as e:
        print('Downloading {} from Quandl'.format(quandl_id))
        df = quandl.get(quandl_id, returns="pandas")
        df.to_pickle(save_path)
        print('Cached {} at {}'.format(quandl_id, save_path))
    return df

In [3]:
exchanges = ['KRAKEN','COINBASE','BITSTAMP','ITBIT']

exchange_data = {}

for exchange in exchanges:
    exchange_code = 'BCHARTS/{}USD'.format(exchange)
    btc_exchange_df = get_quandl_data(exchange_code)
    exchange_data[exchange] = btc_exchange_df

Loaded BCHARTS/KRAKENUSD from cache
Loaded BCHARTS/COINBASEUSD from cache
Loaded BCHARTS/BITSTAMPUSD from cache
Loaded BCHARTS/ITBITUSD from cache


## Dictionary de Data Frames

In [4]:
print(type(exchange_data),'\n')
print(exchange_data['KRAKEN'].head())

<class 'dict'> 

                 Open       High        Low      Close  Volume (BTC)  \
Date                                                                   
2014-01-07  874.67040  892.06753  810.00000  810.00000     15.622378   
2014-01-08  810.00000  899.84281  788.00000  824.98287     19.182756   
2014-01-09  825.56345  870.00000  807.42084  841.86934      8.158335   
2014-01-10  839.99000  857.34056  817.00000  857.33056      8.024510   
2014-01-11  858.20000  918.05471  857.16554  899.84105     18.748285   

            Volume (Currency)  Weighted Price  
Date                                           
2014-01-07       13151.472844      841.835522  
2014-01-08       16097.329584      839.156269  
2014-01-09        6784.249982      831.572913  
2014-01-10        6780.220188      844.938794  
2014-01-11       16698.566929      890.671709  


## Media dos 'Weighted Prices'

In [5]:
def merge_dfs_on_column(dataframes, labels, col):
    '''Merge a single column of each dataframe into a new combined dataframe'''
    series_dict = {}
    for index in range(len(dataframes)):
        series_dict[labels[index]] = dataframes[index][col] # eles tem tamanho diferente, mas sao completados
        
    return pd.DataFrame(series_dict)

In [6]:
print(exchange_data.keys(), '\n')
exchanges_list = list(exchange_data.values()) # lista de data frames
print('Tipo de dados de "exchange_list": ' + str(type(exchanges_list)), '\n')
print('Tipo de dados do primeiro elemento de "exchange_list": ' + str(type(exchanges_list[0])), '\n') # 1a exchange
print(exchanges_list[0].Open[0]) # Primeiro valor de abertura da primeira exchange
print(exchanges_list[0]["High"].head())
exchanges_list[0].head() # Primeiros itens da primeira exchange

dict_keys(['KRAKEN', 'COINBASE', 'BITSTAMP', 'ITBIT']) 

Tipo de dados de "exchange_list": <class 'list'> 

Tipo de dados do primeiro elemento de "exchange_list": <class 'pandas.core.frame.DataFrame'> 

874.6704
Date
2014-01-07    892.06753
2014-01-08    899.84281
2014-01-09    870.00000
2014-01-10    857.34056
2014-01-11    918.05471
Name: High, dtype: float64


,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2014-01-07,874.67040,892.06753,810.00000,810.00000,15.622378,13151.472844,841.835522
2014-01-08,810.00000,899.84281,788.00000,824.98287,19.182756,16097.329584,839.156269
2014-01-09,825.56345,870.00000,807.42084,841.86934,8.158335,6784.249982,831.572913
2014-01-10,839.99000,857.34056,817.00000,857.33056,8.024510,6780.220188,844.938794
2014-01-11,858.20000,918.05471,857.16554,899.84105,18.748285,16698.566929,890.671709


In [7]:
btc_usd_datasets = merge_dfs_on_column(list(exchange_data.values()), list(exchange_data.keys()), 'Weighted Price')

In [8]:
btc_usd_datasets.tail(7)

,BITSTAMP,COINBASE,ITBIT,KRAKEN
Date,,,,
2018-04-03,7335.186319,7314.408376,7348.160038,7343.227556
2018-04-04,7031.668557,7005.337171,7036.746973,7020.550685
2018-04-05,6747.860157,6738.859545,6744.800604,6737.633988
2018-04-06,6639.738385,6641.278312,6660.611024,6627.393905
2018-04-07,6896.336489,6897.141529,6890.366069,6894.374930
2018-04-08,7003.116717,6992.524808,6989.352400,6993.192742
2018-04-09,6841.363821,6856.970973,6886.100183,6846.934177


In [9]:
def df_scatter(df, title, seperate_y_axis=False, y_axis_label='', scale='linear', initial_hide=False):
    '''Generate a scatter plot of the entire dataframe'''
    label_arr = list(df) # lista os nomes das colunas (neste caso, nomes das exchanges)
    series_arr = list(map(lambda col: df[col], label_arr))
    
    layout = go.Layout(
        title=title,
        legend=dict(orientation="h"),
        xaxis=dict(type='date'),
        yaxis=dict(
            title=y_axis_label,
            showticklabels= not seperate_y_axis,
            type=scale
        )
    )
    
    y_axis_config = dict(
        overlaying='y',
        showticklabels=False,
        type=scale )
    
    visibility = 'visible'
    if initial_hide:
        visibility = 'legendonly'
        
    # Form Trace For Each Series
    # Enumerate allows us to loop over something and have an automatic counter.
    trace_arr = []
    for index, series in enumerate(series_arr):
        # index = [0, 1, 2, 3]
        # series = pandas's Series
        # series.index = indices used to access data in the Series
        trace = go.Scatter(
            x=series.index, 
            y=series, 
            name=label_arr[index],
            visible=visibility
        )
        
        # Add seperate axis for the series
        if seperate_y_axis:
            trace['yaxis'] = 'y{}'.format(index + 1)
            layout['yaxis{}'.format(index + 1)] = y_axis_config    
        trace_arr.append(trace)

    fig = go.Figure(data=trace_arr, layout=layout)
    py.iplot(fig)

In [10]:
# Plot all of the BTC exchange prices
df_scatter(btc_usd_datasets, 'Bitcoin Price (USD) By Exchange')

In [11]:
# Remove "0" values
btc_usd_datasets.replace(to_replace=0, value=np.nan, inplace=True)

In [12]:
# Plot the revised dataframe
df_scatter(btc_usd_datasets[['ITBIT','KRAKEN']], 'Bitcoin Price (USD) By Exchange')

In [13]:
# Calculate the average BTC price as a new column
avg_btc_price_usd = btc_usd_datasets.mean(axis=1)

In [14]:
avg_btc_price_usd

Date
2011-09-13       5.929231
2011-09-14       5.590798
2011-09-15       5.094272
2011-09-16       4.854515
2011-09-17       4.870000
2011-09-18       4.839576
2011-09-19       4.900000
2011-09-20       5.388054
2011-09-21       5.706075
2011-09-22       5.696960
2011-09-23       5.707099
2011-09-24       5.722494
2011-09-25       6.050000
2011-09-26       5.983073
2011-09-27       4.895402
2011-09-28       4.862498
2011-09-29       4.816345
2011-09-30            NaN
2011-10-01            NaN
2011-10-02            NaN
2011-10-03       4.847772
2011-10-04       4.924737
2011-10-05       5.029419
2011-10-06       4.908846
2011-10-07       4.687629
2011-10-08       4.455536
2011-10-09       3.890000
2011-10-10       4.498302
2011-10-11       3.998965
2011-10-12       3.943964
                 ...     
2018-03-11    9129.405294
2018-03-12    9330.344791
2018-03-13    9152.211301
2018-03-14    8564.705331
2018-03-15    8067.795078
2018-03-16    8301.331495
2018-03-17    8039.581190
2018-03

In [15]:
btc_trace = go.Scatter(x=avg_btc_price_usd.index, y=avg_btc_price_usd)
py.iplot([btc_trace])

In [16]:
avg_btc_price_usd.index[avg_btc_price_usd.isnull()]

DatetimeIndex(['2011-09-30', '2011-10-01', '2011-10-02', '2011-10-15',
               '2011-10-16', '2011-10-18', '2011-10-19', '2011-10-22',
               '2011-10-23', '2011-10-27', '2011-11-02', '2011-11-03',
               '2011-11-07', '2011-11-23', '2011-11-27', '2011-12-04',
               '2011-12-09', '2011-12-17'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [17]:
btc_price_usd = avg_btc_price_usd.loc['2012-01-01':]

In [18]:
btc_trace = go.Scatter(x=btc_price_usd.index, y=btc_price_usd) # btc_trace e' tipo um dicionario
py.iplot([btc_trace])

In [19]:
save_path = '../Data/Bitcoin_price_daily.pkl'
btc_price_usd.to_pickle(save_path)

Medias dos demais valores
----
IMPORTANTE - exchange "imaginaria":
- Os volumes sao somados, e os demais valores averaged, i.e., todas as exchanges tem peso igual

In [20]:
exchange_data['BITSTAMP']

,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2011-09-13,5.80,6.00,5.65,5.97,58.371382,3.460974e+02,5.929231
2011-09-14,5.58,5.72,5.52,5.53,61.145984,3.418548e+02,5.590798
2011-09-15,5.12,5.24,5.00,5.13,80.140795,4.082590e+02,5.094272
2011-09-16,4.82,4.87,4.80,4.85,39.914007,1.937631e+02,4.854515
2011-09-17,4.87,4.87,4.87,4.87,0.300000,1.461000e+00,4.870000
2011-09-18,4.87,4.92,4.81,4.92,119.812800,5.798431e+02,4.839576
2011-09-19,4.90,4.90,4.90,4.90,20.000000,9.800000e+01,4.900000
2011-09-20,4.92,5.66,4.92,5.66,89.280711,4.810493e+02,5.388054
2011-09-21,5.70,5.79,5.66,5.66,17.629322,1.005942e+02,5.706075


In [21]:
def mergewithfunction(exchanges_dict, function_dict):
    '''Une os indices <function_dict.keys> das exchanges de acordo com as funcoes <function_dict.values> '''
    
    dict_joined_indices = {} # dicionario dos indices agregados das exchanges
    for market_index_name in function_dict.keys(): # para cada indice do mercado, e.g., high, low, volume, etc
        dict_index_exchange = {}
        for exchange_name in exchanges_dict.keys(): # para cada exchange, e.g., COINBASE, KRAKEN, etc
            dict_index_exchange[exchange_name] = exchanges_dict[exchange_name][market_index_name]
        market_index_df = pd.DataFrame(data = dict_index_exchange)
        dict_joined_indices[market_index_name] = market_index_df.apply(function_dict[market_index_name], axis=1)
    market_df = pd.DataFrame(data = dict_joined_indices)
    return market_df

In [22]:
index_func_dict = {
    'Open': np.mean,
    'High': np.mean, 
    'Low': np.mean, 
    'Close': np.mean,
    'Volume (BTC)': np.sum, 
    'Volume (Currency)': np.sum,
    'Weighted Price': np.mean
}
bitcoin_data = mergewithfunction(exchange_data, index_func_dict)

In [23]:
bitcoin_data = bitcoin_data.loc['2012-01-01':]

In [24]:
save_path = '../Data/Bitcoin_data.pkl'
bitcoin_data.to_pickle(save_path)

In [25]:
bitcoin_data

,Close,High,Low,Open,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2012-01-01,5.0000,5.0000,4.5800,4.5800,21.602000,1.057792e+02,4.896730
2012-01-02,5.0000,5.0000,5.0000,5.0000,19.048000,9.524000e+01,5.000000
2012-01-03,5.2900,5.3200,5.1400,5.3200,88.037281,4.648052e+02,5.279641
2012-01-04,5.5700,5.5700,4.9300,4.9300,107.233260,5.680762e+02,5.297575
2012-01-05,6.6500,6.6500,5.7200,5.7200,94.801829,5.877348e+02,6.199614
2012-01-06,6.0000,6.9000,6.0000,6.6500,33.882747,2.228052e+02,6.575772
2012-01-07,6.8000,6.8000,6.8000,6.8000,0.295858,2.011834e+00,6.800000
2012-01-08,7.0000,7.0000,6.9000,6.9000,5.000000,3.460000e+01,6.920000
2012-01-09,6.3000,6.9900,6.2300,6.9900,66.869323,4.430447e+02,6.625530


In [26]:
btc_open_trace = go.Scatter(x=bitcoin_data.index, y=bitcoin_data['Open'], name='Open')
btc_high_trace = go.Scatter(x=bitcoin_data.index, y=bitcoin_data['High'], name='High')
btc_low_trace = go.Scatter(x=bitcoin_data.index, y=bitcoin_data['Low'], name='Low')
btc_close_trace = go.Scatter(x=bitcoin_data.index, y=bitcoin_data['Close'], name='Close')
btc_volbtc_trace = go.Scatter(x=bitcoin_data.index, y=bitcoin_data['Volume (BTC)'], name='Volume (BTC)')
btc_volcur_trace = go.Scatter(x=bitcoin_data.index, y=bitcoin_data['Volume (Currency)'], name='Volume (Currency)')
btc_price_trace = go.Scatter(x=bitcoin_data.index, y=bitcoin_data['Weighted Price'], name='Weighted Price')

In [27]:
from plotly import tools
# layout = go.Layout(
#     title='Bitcoin Data',
#     legend=dict(orientation='h'),
#     xaxis=dict(
#         title='Data',
#         titlefont=dict(
#             family='Courier New, monospace',
#             size=18,
#             color='#7f7f7f'
#         )
#     ),
#     yaxis=dict(
#         title='Preco',
#         titlefont=dict(
#             family='Courier New, monospace',
#             size=18,
#             color='#7f7f7f'
#         )
#     )
# )
fig = tools.make_subplots(rows=3, cols=1)
fig.append_trace(btc_open_trace, 1, 1)
fig.append_trace(btc_high_trace, 1, 1)
fig.append_trace(btc_low_trace, 1, 1)
fig.append_trace(btc_close_trace, 1, 1)
fig.append_trace(btc_price_trace, 1, 1)
fig.append_trace(btc_volbtc_trace, 2, 1)
fig.append_trace(btc_volcur_trace, 3, 1)
fig.layout.update(title='Bitcoin Data', height=1000, legend=dict(orientation='h'))
py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]

